<a href="https://colab.research.google.com/github/telmacarvalho/pi-4-univesp/blob/main/inferencia_ml_pi_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferência dos resultados do modelo

A solução adotada é uma técnica chamada **previsão iterativa (ou recursiva)**. A lógica é a seguinte:

* Usamos os dados reais de 2023 para prever 2024.
* Como não temos os dados reais de 2024, usamos a própria previsão que o modelo gerou para 2024 como se fossem os dados de entrada para prever 2025.
* Repetimos o processo: usamos a previsão de 2025 como entrada para prever 2026.

## Importação de bibliotecas

In [1]:
import pandas as pd
import numpy as np
import joblib
import os
from google.colab import drive

##  14. Importação do modelo e dos dados para previsões

In [2]:
# Autoriza acesso ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Carrega o modelo
caminho_drive = '/content/drive/MyDrive/dados/'
caminho_modelo = f'{caminho_drive}output/modelo_random_forest_multi.joblib'
modelo_carregado = joblib.load(caminho_modelo)
print("Modelo final carregado com sucesso!")

Modelo final carregado com sucesso!


In [4]:
# Carrega os dados de 2023
caminho_dados_processados = f'{caminho_drive}csv/input/data_inference.csv'
dados_2023 = pd.read_csv(caminho_dados_processados)
dados_2023

,cnae,total_acidentes,com_cat_registrada,acidentes_tipicos,acidentes_trajeto,doenca_trabalho,sem_cat_registrada,ano
0,910,490,450,357,62,31,40,2023
1,1921,265,205,111,72,22,60,2023
2,2513,459,434,378,51,5,25,2023
3,3321,897,813,698,110,5,84,2023
4,3520,56,50,27,23,0,6,2023
5,4223,50,46,39,6,1,4,2023
6,4292,2441,2236,1961,233,42,205,2023
7,4321,4345,4012,3089,889,34,333,2023
8,4940,43,41,19,22,0,2,2023
9,7112,5193,4895,3853,904,138,298,2023


## 15. Prepara os dados de entrada e define as colunas

In [5]:
# Lista de colunas de alvos que o modelo irá prever
colunas_alvo = [
    'total_acidentes',
    'acidentes_tipicos',
    'acidentes_trajeto',
    'doenca_trabalho'
]

In [6]:
# Lista de colunas de features que o modelo espera na ordem correta
colunas_features = colunas_alvo

In [7]:
# Pega a lista de CNAEs únicos do arquivo de 2023
unique_cnaes = dados_2023['cnae'].unique()
unique_cnaes

array([ 910, 1921, 2513, 3321, 3520, 4223, 4292, 4321, 4940, 7112])

## 16. Executa a previsão interativa

In [8]:
# Lista para armazenar os DataFrames de resultado
lista_de_resultados = []
anos_para_prever = [2024, 2025, 2026]

print(f"\nIniciando a previsão iterativa para {len(unique_cnaes)} CNAEs...\n")

# Loop principal: itera sobre cada CNAE
for cnae in unique_cnaes:
    # Seleciona a linha de dados de 2023 para o CNAE atual
    input_inicial = dados_2023[dados_2023['cnae'] == cnae][colunas_features]

    input_atual = input_inicial.copy()

    print(f"\n -- Previsão para o CNAE {cnae} iniciada: ")

    # Loop secundário: itera sobre os anos que queremos prever
    for ano in anos_para_prever:
        # Faz a previsão para o ano e CNAE atuais
        previsao_array = modelo_carregado.predict(input_atual)

        # Formata o resultado da previsão em um DataFrame
        resultado_ano = pd.DataFrame(np.round(previsao_array).astype(int), columns=colunas_alvo)
        resultado_ano['cnae'] = cnae
        resultado_ano['ano_previsto'] = ano

        # Adiciona o resultado formatado à lista principal
        lista_de_resultados.append(resultado_ano)

        # Prepara o input para a próxima iteração (próximo ano)
        previsao_df = pd.DataFrame(previsao_array, columns=colunas_alvo)
        # O input para o próximo ano será a previsão que acabou de ser feita
        input_atual = previsao_df[colunas_features]

        print(f"\nPrevisão para o ano {ano} finalizada.")

print("\nPrevisão finalizada para todos os CNAEs.")


Iniciando a previsão iterativa para 10 CNAEs...


 -- Previsão para o CNAE 910 iniciada: 

Previsão para o ano 2024 finalizada.

Previsão para o ano 2025 finalizada.

Previsão para o ano 2026 finalizada.

 -- Previsão para o CNAE 1921 iniciada: 

Previsão para o ano 2024 finalizada.

Previsão para o ano 2025 finalizada.

Previsão para o ano 2026 finalizada.

 -- Previsão para o CNAE 2513 iniciada: 

Previsão para o ano 2024 finalizada.

Previsão para o ano 2025 finalizada.

Previsão para o ano 2026 finalizada.

 -- Previsão para o CNAE 3321 iniciada: 

Previsão para o ano 2024 finalizada.

Previsão para o ano 2025 finalizada.

Previsão para o ano 2026 finalizada.

 -- Previsão para o CNAE 3520 iniciada: 

Previsão para o ano 2024 finalizada.

Previsão para o ano 2025 finalizada.

Previsão para o ano 2026 finalizada.

 -- Previsão para o CNAE 4223 iniciada: 

Previsão para o ano 2024 finalizada.

Previsão para o ano 2025 finalizada.

Previsão para o ano 2026 finalizada.

 -- Previsão p

# 17. Consolidação e apresentação dos resultados

In [9]:
# Concatena todos os pequenos df da lista em um único DataFrame final
df_previsoes_final = pd.concat(lista_de_resultados, ignore_index=True)
df_previsoes_final.head()

,total_acidentes,acidentes_tipicos,acidentes_trajeto,doenca_trabalho,cnae,ano_previsto
0,463,362,59,11,910,2024
1,463,367,51,14,910,2025
2,462,362,54,13,910,2026
3,327,220,52,14,1921,2024
4,285,211,36,11,1921,2025


In [10]:
# Reorganiza as colunas para uma melhor visualização
ordem_colunas = ['cnae', 'ano_previsto'] + colunas_alvo
df_previsoes_final = df_previsoes_final[ordem_colunas]
df_previsoes_final.head()

,cnae,ano_previsto,total_acidentes,acidentes_tipicos,acidentes_trajeto,doenca_trabalho
0,910,2024,463,362,59,11
1,910,2025,463,367,51,14
2,910,2026,462,362,54,13
3,1921,2024,327,220,52,14
4,1921,2025,285,211,36,11


In [11]:
df_previsoes_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   cnae               30 non-null     int64
 1   ano_previsto       30 non-null     int64
 2   total_acidentes    30 non-null     int64
 3   acidentes_tipicos  30 non-null     int64
 4   acidentes_trajeto  30 non-null     int64
 5   doenca_trabalho    30 non-null     int64
dtypes: int64(6)
memory usage: 1.5 KB


## 18. Amazenamento no drive

In [17]:
# Salvar os resultados em arquivo CSV
caminho_previsoes_cnae = os.path.join(caminho_drive, 'output/previsoes_2024_2025_2026.csv')
df_previsoes_final.to_csv(caminho_previsoes_cnae, index=False)
print(f"\nArquivo com as projeções por CNAE foi salvo em: {caminho_previsoes_cnae}")


Arquivo com as projeções por CNAE foi salvo em: /content/drive/MyDrive/dados/output/previsoes_2024_2025_2026.csv


In [18]:
# Salvar os resultados em arquivo excel
caminho_previsoes_cnae = os.path.join(caminho_drive, 'output/previsoes_2024_2025_2026.xlsx')
df_previsoes_final.to_excel(caminho_previsoes_cnae, index=False)
print(f"\nArquivo com as projeções por CNAE foi salvo em: {caminho_previsoes_cnae}")


Arquivo com as projeções por CNAE foi salvo em: /content/drive/MyDrive/dados/output/previsoes_2024_2025_2026.xlsx
